**3a.** based on your implementation of the EM algorithm from Exercise 3, but leaving out the median
filtering, create a discrete (hard / non-probabilistic) label image that contains the most likely
material for each pixel. Output it as an RGB image. (1P)

In [15]:
# Implementation of gaussian mixture model with EM-algorithm and clustering for MR imaging
# from Task 2

import cv2
import numpy as np
import matplotlib.pyplot as plt
from scipy.stats import norm

class GMM:
    
    def __init__(self, mix_coeff: np.ndarray, sigma: np.ndarray, mean: np.ndarray, n: int, k: int):
        self.k = k 
        self.n = n 
        self.sigma = sigma                  # sigma: k
        self.mean = mean                    # mean: k 
        self.mix_coeff = mix_coeff          # mixture coefficents: k 
        self.prob = np.zeros(shape=(n,k))   # probabilites: n x k
    
    # Helper function to compute gaussians    
    def gaussian(self, x, mean, sigma):
        return norm.pdf(x, loc=mean, scale=sigma)

    # Expectation step (first step of EM-Algorithm)
    def e_step(self, data: np.ndarray):
        data = data.astype(np.float32)
        K = self.k         
        N = len(data)
        prob = np.zeros((N, K), dtype=np.float32)
        
        for k in range(K):
            prob[:, k] = self.gaussian(data, self.mean[k], self.sigma[k]) * self.mix_coeff[k]
        denom = prob.sum(axis=1, keepdims=True)
        self.prob = prob / denom
        self.n = N
    
    # Maximization step (second step of EM-Algorithm)
    def m_step(self, data: np.ndarray):
        data = data.astype(np.float32)
        K = self.k 
        
        # Effective number
        N_k = np.sum(self.prob, axis=0)
        
        # Mixture coefficents
        mix_coeff = N_k / self.n
        
        # Mean
        mean = self.prob * data[:, None]
        mean = np.sum(mean, axis=0)        
        mean /= N_k
        
        # Sigma
        diff2 = (np.outer(data, np.ones(K)) - np.outer(np.ones_like(data), mean))**2   
        sigma2 = self.prob * diff2 
        sigma2 = np.sum(sigma2, axis=0)
        sigma2 /= N_k
        var_floor = 1e-4
        sigma2 = np.maximum(sigma2, var_floor)
        sigma = np.sqrt(sigma2)
                
        # Update 
        self.sigma = sigma
        self.mean = mean   
        self.mix_coeff = mix_coeff            
    
    # Run EM-Algorithm for given number of iterations
    def run_em_algo(self, data: np.ndarray, max_iters: int):
        for _ in range(max_iters):
            self.e_step(data)
            self.m_step(data)
    
    # Predict probability for new data point
    def predict(self, data: float):
        data = data.astype(np.float32)
        prob = np.zeros(self.k)
        K = self.k
        for k in range(K):
            mean = self.mean[k]
            sigma = self.sigma[k]
            mix_coeff = self.mix_coeff[k]
            prob[k] = self.gaussian(x=data, mean=mean, sigma=sigma) * mix_coeff
        sum = np.sum(prob, axis=0)
        prob /= sum
        return prob
    
    
    # Clustering for given image and mask, return segmented image
    def viz(self, img: np.ndarray, mask: np.ndarray, discrete_labeling = False):
        segmented_img = np.zeros(shape=np.hstack([img.shape,3]), dtype=np.uint8)
        for y in np.arange(img.shape[0]):
            for x in np.arange(img.shape[1]):
                if mask[y,x] > 0:
                    data_x = img[y,x]
                    prob = self.predict(data_x)
                    prob2 = np.array([prob[1], prob[2], prob[0]])
                    if discrete_labeling:
                        try:
                            label_idx = np.argmax(prob2)
                            prob2 = np.zeros_like(prob2)
                            prob2[label_idx] = 1
                            prob2 = prob2.astype(np.uint8)
                        except RuntimeWarning:
                            print(label_idx)
                    segmented_img[y,x] = (255*prob2).astype(np.uint8)
        return segmented_img

In [18]:
# Import images 
mask_file = 'mask.png'
img_file = 'brain-noisy.png'

mask = cv2.imread(mask_file, cv2.IMREAD_GRAYSCALE)
img = cv2.imread(img_file, cv2.IMREAD_GRAYSCALE)

# Create mask
mask = (mask>0).astype(np.uint8)

# Initialization GMM parameters (same from task 2)
data = img[mask > 0].astype(np.float32)
sigma = np.array([20,19,20])
mean = np.array([75, 160, 225])
mix_coeff = np.array([0.14, 0.37, 0.49])

# Run EM algorithm with many iterations. Afterwards visualize the clustering output 
# as segmented image with discrete hard labeling. 
gmm = GMM(mix_coeff, sigma, mean, len(data), len(mix_coeff))
iters = 1000
gmm.run_em_algo(data, iters)
segmented_img_1 = gmm.viz(img, mask, discrete_labeling=True)
cv2.imshow(f'Segmented noisy Image after {iters} iterations', segmented_img_1)
cv2.waitKey(0)
cv2.destroyAllWindows()   
cv2.imwrite('task3_segmented_image_gmm_noisy.png', segmented_img_1)

True

**Image: Noisy GMM Output with discrete Labeling**

File: task3_segmented_image_gmm_noisy.png

![task3_segmented_image_gmm_noisy.png](task3_segmented_image_gmm_noisy.png)